In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import csv
import math
import matplotlib.pyplot as plt

In [ ]:
!pip install python-louvain
import community.community_louvain as cl

In [ ]:
import community
import matplotlib.cm as cm

## Добавление сервисов


In [ ]:
# Чтение данных из файла или создание данных
data = pd.read_csv('infrastructure_nao.csv', skiprows=[0], delimiter=';', names=['name','school','kindergarden','post','library','culture','atm','health','port','airport','shop', 'pvz', 'gas', 'mfc'])

df_inf = pd.DataFrame(data)

# Нормализация данных

def normalize_columns(df_inf, columns=None):
    if not columns:
        columns = df_inf.columns[1:] # все колонки кроме первой

    for col in columns:
        max_value = float(df_inf[col].max())
        min_value = float(df_inf[col].min())

        df_inf[col] = (df_inf[col].astype('float') - min_value) / (max_value - min_value)
    return df_inf


# Нормализуем данные
normalized_df_inf = normalize_columns(df_inf)

# Суммируем нормализованные значения
summed_values = normalized_df_inf.iloc[:, 1:].sum(axis=1) #Все строки и колонки кроме первых

# Добавляем итоговую сумму к исходному DataFrame
result_df_inf = df_inf.copy()
result_df_inf['ind'] = summed_values

result_df_inf = result_df_inf[[result_df_inf.columns[0], result_df_inf.columns[-1]]] #Выбираем только столбцы с названием и итоговой оценкой

df_towns = pd.read_csv('main_towns.csv', delimiter = ';') #Читаем файл с опорными городами
combined_df = pd.concat([result_df_inf, df_towns])

# Запись результата в файл
#combined_df.to_csv("result_inf_1.csv", sep=';', index=False)

#print(combined_df)
combined_df.sort_values(by='ind', ascending=False)

,name,ind
6,Новый Уренгой,15.000000
7,Норильск,15.000000
3,Якутск,15.000000
4,Магадан,15.000000
9,Красноярск,15.000000
1,Архангельск,15.000000
0,Нарьян-Мар,14.000000
2,Воркута,14.000000
10,Усинск,14.000000
5,Тарко-Сале,13.000000


In [ ]:
data

,name,school,kindergarden,post,library,culture,atm,health,port,airport,shop,pvz,gas,mfc
0,Великовисочное,1,0,1,1,1,1,2,1,1,8,0,0,1
1,Усть-Кара,1,1,1,1,1,0,1,1,1,4,0,0,0
2,Коткино,1,0,1,1,1,0,1,1,1,4,0,0,1
3,Щелино,0,0,0,1,1,0,1,1,1,1,0,0,0
4,Каменка,0,0,1,1,0,0,1,1,1,1,0,0,0
5,Лабожское,0,0,1,1,1,0,1,1,1,3,0,0,0
6,Хонгурей,0,0,1,1,1,0,1,1,1,3,0,0,0
7,Оксино,1,0,1,1,1,0,2,1,1,4,0,0,1
8,Тельвиска,1,1,1,1,1,0,1,1,0,2,0,0,1
9,Макарово,0,0,1,1,1,0,1,1,0,2,0,0,0


In [ ]:
df = pd.DataFrame(data)

# Подсчёт количества нулей в каждом столбце
zero_count = df.apply(lambda x: (x == 0).sum())

zero_count

,0
name,0
school,22
kindergarden,28
post,9
library,8
culture,8
atm,36
health,5
port,13
airport,13


Расчёт функции времени (T). Виды транспорта:
1. Автомобильный (car)
2. Авиационный (plane)
3. Водный_теплоход (water_ship)
4. Водный_личный (water_boat)
5. Зимний (winter)

2 времени года лето (warm), зима (cold). Для каждого отдельный расчёт. Коэффициенты риска меняются (задаются самостоятельно, отдельно для каждого эксперимента)

T = t * an
an - коэффициент риска транспорта (ratio)


## **Расчёт тёплого времени года**

In [ ]:
#Данные на всякий случай сохранить в кодировке utf-8 (через блокнот)
car_ratio=1.1
plane_ratio=1.2
water_ship_ratio=1.2
water_boat_ratio=1.3

# Чтение данных из CSV-файлаw
df = pd.read_csv("warm_time_nao.csv", delimiter=';', names=['depart','arrival','car','plane','water_ship','water_boat']) #Перед этим в экселе установить числовой формат и убрвть первую строку с названиями столбцов

#Замена , на .
df['car'] = df['car'].apply(lambda x: x.replace(',', '.')).astype('float')
df['plane'] = df['plane'].apply(lambda x: x.replace(',', '.')).astype('float')
df['water_ship'] = df['water_ship'].apply(lambda x: x.replace(',', '.')).astype('float')
df['water_boat'] = df['water_boat'].apply(lambda x: x.replace(',', '.')).astype('float')
print(df)

#Умножение по столбцам на коэффициенты риска
df['car'] = df['car'].apply(lambda x: car_ratio*x)
df['plane'] = df['plane'].apply(lambda x: plane_ratio*x)
df['water_ship'] = df['water_ship'].apply(lambda x: water_ship_ratio*x)
df['water_boat'] = df['water_boat'].apply(lambda x: water_boat_ratio*x)

# Запись результата в файл
#df.to_csv("result_time.csv", sep=';', index=False)

          depart      arrival   car  plane  water_ship  water_boat
0          Андег  Нельмин-Нос   0.0   0.98        0.83        0.83
1          Андег          Куя   0.0   0.00        0.75        0.75
2    Архангельск   Нарьян-Мар  46.0   1.25        0.00        0.00
3    Архангельск         Несь   0.0   2.00        0.00        0.00
4    Архангельск         Чижа   0.0   2.33        0.00        0.00
..           ...          ...   ...    ...         ...         ...
122       Харута    Хорей-Вер   0.0   0.63        0.00        0.00
123     Хонгурей      Каменка   0.5   0.00        0.00        0.00
124         Чижа       Индига   0.0   1.29        0.00        0.00
125        Шойна         Чижа   0.0   0.44        0.00        0.00
126       Щелино      Коткино   0.0   0.00        4.03        2.00

[127 rows x 6 columns]


In [ ]:
# Чтение данных из файла
df_time = df

# Получение минимальных значений, игнорируя 0 и первые два столбца
results = df.iloc[:, 2:].replace(0, np.nan).apply(lambda x: x.min(), axis=1)

#Сохранение результатов в таблицу
new_df = df_time.copy()
new_df['min'] = results

#Выбор столбцов с названием н.п. и минимальным временем
min_time_warm = new_df[['depart', 'arrival','min']]

#Заполнение пустых строк нулями
min_time_warm['min'] = min_time_warm['min'].fillna(0)

# Печать результатов и их сохранение в файл
min_time_warm

<ipython-input-10-ee1f91098911>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_time_warm['min'] = min_time_warm['min'].fillna(0)


,depart,arrival,min
0,Андег,Нельмин-Нос,0.996
1,Андег,Куя,0.900
2,Архангельск,Нарьян-Мар,1.500
3,Архангельск,Несь,2.400
4,Архангельск,Чижа,2.796
...,...,...,...
122,Харута,Хорей-Вер,0.756
123,Хонгурей,Каменка,0.550
124,Чижа,Индига,1.548
125,Шойна,Чижа,0.528


## Расчёт гравитационной силы (G)

In [ ]:
# Чтение файлов с уровнем инфраструктурной обеспеченности и данными по времени
settl = dict(zip(combined_df['name'], combined_df['ind']))

# Получение данных из min_time_warm
connect = min_time_warm.values.tolist()

# Вычисления и создание нового датафрейма
Gravity = []
for row in connect:
    try:
        F1 = float(settl[row[0]])  # Значение F1 для первого населенного пункта
        F2 = float(settl[row[1]])  # Значение F2 для второго населенного пункта
        r = float(row[2])          # Расстояние между пунктами
        if r == 0:
            LocalGravity = 0
        else:
            LocalGravity = (F1 - F2) / (r ** 2)  # Рассчитываем LocalGravity
        Gravity.append([
            row[0],           # Название первого населенного пункта
            row[1],           # Название второго населенного пункта
            LocalGravity,
            F1 - F2,          # Разница между F1 и F2
            row[2],           # Время из столбца min_time_warm
            F1,               # Добавление F1
            F2                # Добавление F2
        ])                    # Добавляем F1-F2 и r
    except KeyError:
        continue  # Пропускаем строки, если ключ не найден

# Создаем новый датафрейм с нужными столбцами
local_gravity = pd.DataFrame(
    Gravity,
    columns=[
        'source',             # Название первого населенного пункта
        'recipient',          # Название второго населенного пункта
        'LocalGravity',
        'F1-F2',              # Разница между F1 и F2
        'time',               # Время из столбца min_time_warm
        'F1',                 # Значение F1 для деревни source
        'F2'                  # Значение F2 для деревни recipient
    ]
)

# Сортируем по LocalGravity в порядке убывания
local_gravity.sort_values(by='LocalGravity', ascending=False)

,source,recipient,LocalGravity,F1-F2,time,F1,F2
56,Нарьян-Мар,Тельвиска,427.350427,7.222222,0.130,14.000000,6.777778
59,Нарьян-Мар,Андег,353.842501,8.611111,0.156,14.000000,5.388889
107,Ома,Снопа,133.959191,2.777778,0.144,7.111111,4.333333
57,Нарьян-Мар,Макарово,128.212582,9.555556,0.273,14.000000,4.444444
106,Ома,Вижас,111.859888,2.722222,0.156,7.111111,4.388889
...,...,...,...,...,...,...,...
35,Красное,Искателей,-2.366065,-3.277778,1.177,8.722222,12.000000
114,Пылемец,Тельвиска,-2.669400,-2.388889,0.946,4.388889,6.777778
32,Кия,Чижа,-20.721812,-3.055556,0.384,1.333333,4.388889
22,Волонга,Белушье,-37.399219,-1.055556,0.168,3.388889,4.444444


In [ ]:
# Чтение CSV файла
df_row = local_gravity

# Функция для обработки строк
def process_row(row):
    if row['LocalGravity'] < 0:
        # Меняем города местами и делаем значение положительным
        row['source'], row['recipient'] = row['recipient'], row['source']
        row['LocalGravity'] = abs(row['LocalGravity'])

    # Проверка изменения знака в F1-F2
    if row['F1-F2'] < 0:
        # Меняем значения F1 и F2 местами
        row['F1'], row['F2'] = row['F2'], row['F1']
        # Делаем значение F1-F2 положительным
        row['F1-F2'] = abs(row['F1-F2'])

    return row

# Применяем функцию ко всем строкам
df_row = df_row.apply(process_row, axis=1)
df_row.sort_values(by='LocalGravity', ascending=False)

# Отображение изменённого датафрейма
df_row

,source,recipient,LocalGravity,F1-F2,time,F1,F2
0,Нельмин-Нос,Андег,1.904091,1.888889,0.996,7.277778,5.388889
1,Андег,Куя,2.469136,2.000000,0.900,5.388889,3.388889
2,Архангельск,Нарьян-Мар,0.444444,1.000000,1.500,15.000000,14.000000
3,Архангельск,Несь,1.176698,6.777778,2.400,15.000000,8.222222
4,Архангельск,Чижа,1.357333,10.611111,2.796,15.000000,4.388889
...,...,...,...,...,...,...,...
122,Харута,Хорей-Вер,0.583224,0.333333,0.756,7.444444,7.111111
123,Хонгурей,Каменка,3.673095,1.111111,0.550,5.500000,4.388889
124,Индига,Чижа,1.136009,2.722222,1.548,7.111111,4.388889
125,Шойна,Чижа,7.373291,2.055556,0.528,6.444444,4.388889


In [ ]:
anchor_points = ['Несь', 'Нижняя Пеша', 'Оксино', 'Лабожское', 'Великовисочное', 'Чижа', 'Усть-Кара', 'Хорей-Вер', 'Волоковая']  # Список опорных пунктов

# Нормализация разницы F1-F2 и времени связности
df_row['normalized_f_diff'] = (df_row['F1-F2'] - df_row['F1-F2'].min()) / (df_row['F1-F2'].max() - df_row['F1-F2'].min())

# Обратная нормализация времени (минимум = 1, максимум = 0)
df_row['normalized_time'] = 1 - ((df_row['time'] - df_row['time'].min()) / (df_row['time'].max() - df_row['time'].min()))

# Настройки порогов для классификации
f_diff_threshold = 0.4  # Порог для разницы F1-F2 (нормированное значение)
time_threshold = 0.6  # Порог для времени связности (нормированное значение)

# Классификация ситуаций
df_row['action'] = ''

for index, row in df_row.iterrows():
    f_diff = row['normalized_f_diff']
    time = row['normalized_time']

    if row['recipient'] in anchor_points:
        action = 'Приоритетная связь'  # Связь с опорным пунктом имеет наивысший приоритет
    elif f_diff > f_diff_threshold and time > time_threshold:
        action = 'Добавить сервисы'  # Улучшение инфраструктуры рекомендуется, если разница F1-F2 велика и время связи малое
    elif f_diff < f_diff_threshold and time < time_threshold:
        action = 'Сократить время'  # Сокращение времени путешествия рекомендуется, если разница F1-F2 мала и время связи велико
    elif f_diff >= f_diff_threshold and time <= time_threshold:
        action = 'Добавить сервисы и сократить время'  # Сокращение времени путешествия рекомендуется, если разница F1-F2 велика и время связи велико
    else:
        action = 'Никаких действий не требуется'  # Никаких действий не требуется, если разница F1-F2 и время связи находятся в приемлемых пределах

    df_row.at[index, 'action'] = action

# Удаляем лишние столбцы и оставляем только recipient, source и action
final_dataframe = df_row[['recipient', 'source', 'action']]

# Сохранение отчета
#final_dataframe.to_csv('recommendations_final.csv', index=False)

In [ ]:
# Фильтруем строки по нужным условиям
filtered_df_time = final_dataframe[(final_dataframe['recipient'] == 'Индига')]

filtered_df_time

,recipient,source,action
7,Индига,Архангельск,Добавить сервисы
54,Индига,Мезень,Никаких действий не требуется
68,Индига,Нарьян-Мар,Добавить сервисы
91,Индига,Несь,Никаких действий не требуется
93,Индига,Нижняя Пеша,Никаких действий не требуется
109,Индига,Ома,Никаких действий не требуется


In [ ]:
# Группировка по recipient и action с подсчетом количества строк
counted_df = final_dataframe.groupby(['recipient', 'action']).size().reset_index(name='count')

final_dataframe = counted_df.sort_values(by='count', ascending=False)
final_dataframe

,recipient,action,count
53,Чижа,Приоритетная связь,8
38,Пылемец,Никаких действий не требуется,6
56,Щелино,Никаких действий не требуется,5
23,Лабожское,Приоритетная связь,5
14,Каменка,Никаких действий не требуется,5
25,Макарово,Никаких действий не требуется,4
11,Индига,Никаких действий не требуется,4
7,Вижас,Никаких действий не требуется,4
45,Усть-Кара,Приоритетная связь,4
44,Тошвиска,Никаких действий не требуется,4


In [ ]:
# Фильтруем строки по нужным условиям
filtered_df = final_dataframe[(final_dataframe['action'] == 'Приоритетная связь') | (final_dataframe['action'] == 'Добавить сервисы и сократить время')| (final_dataframe['action'] == 'Сократить время')]

filtered_df

,recipient,action,count
53,Чижа,Приоритетная связь,8
23,Лабожское,Приоритетная связь,5
45,Усть-Кара,Приоритетная связь,4
33,Нижняя Пеша,Приоритетная связь,3
32,Несь,Приоритетная связь,3
52,Хорей-Вер,Приоритетная связь,2
34,Оксино,Приоритетная связь,2
4,Великовисочное,Приоритетная связь,1
19,Красное,Добавить сервисы и сократить время,1
28,Мезень,Сократить время,1


### Добавление сервисов

In [ ]:
# Фильтрация строк по названию деревни
selected_villages = ['Харьягинский', 'Устье', 'Вижас', 'Каратайка', 'Индига', 'Шойна', 'Амдерма', 'Пылемец', 'Ома']
filtered_data = data[data['name'].isin(selected_villages)]
filtered_data

,name,school,kindergarden,post,library,culture,atm,health,port,airport,shop,pvz,gas,mfc
17,Вижас,0,0,1,1,1,0,1,0,1,1,0,0,0
20,Ома,1,1,1,1,1,0,2,1,1,2,0,0,0
22,Пылемец,0,0,1,0,1,0,1,1,1,1,0,0,0
23,Устье,0,0,0,0,0,0,0,1,0,1,0,0,0
24,Индига,1,1,1,1,1,0,2,1,1,2,0,0,0
30,Шойна,1,0,1,1,1,0,1,1,1,2,0,0,0
34,Каратайка,1,1,1,1,1,1,2,0,1,5,0,0,1
39,Харьягинский,0,0,0,0,0,0,0,0,0,0,0,1,0
40,Амдерма,1,0,1,1,1,0,1,1,1,2,0,0,1


In [ ]:
# Функция для обновления значений в нескольких столбцах для нескольких деревень
def update_columns_multiple_villages(df, villages_and_updates):
    df_copy = df.copy()  # создаем копию датафрейма

    for village_name, updates in villages_and_updates.items():
        index = df_copy['name'] == village_name  # находим индекс строки с нужной деревней

        for column_name, value in updates.items():
            df_copy.loc[index, column_name] = value  # обновляем значения в указанных столбцах

    return df_copy

In [ ]:
villages_and_updates = {
    'Харьягинский': {'health': 1, 'atm': 1, 'shop': 1, 'post':1, 'culture':1},  #Указываем какие сервисы в каких поселениях добавить
    'Вижас': {'kindergarden': 1, 'gas': 1, 'atm': 1},
    'Устье': {'health': 1, 'post': 1, 'culture': 1, 'port': 1},
    'Каратайка': {'health': 2, 'gas': 1},
    'Шойна': {'kindergarden':1, 'health':2, 'gas':1},
    'Индига': {'shop':3, 'gas':1},
    'Амдерма': {'kindergarden':1, 'atm':1, 'health':2},
    'Пылемец': {'atm':1, 'library':1},
    'Ома': {'atm':1, 'gas':1}
    #Указываем какие сервисы в каких поселениях добавить
}

# Применяем функцию
updated_data = update_columns_multiple_villages(data, villages_and_updates)
#updated_data

### Добавление транспортных связей

In [ ]:
# Копируем датафрейм для внесения изменений
modified_min_time_warm = min_time_warm.copy()

def replace_values_in_multiple_rows(df, rows_indices, column_name, new_values):
    """
    Функция для замены значений в нескольких строках одного столбца.

    :param df: Датафрейм, в котором будут производиться изменения.
    :param rows_indices: Список индексов строк, в которых нужно произвести замену.
    :param column_name: Название столбца, в котором нужно заменить значения.
    :param new_values: Список новых значений для замены. Должен иметь ту же длину, что и rows_indices.
    """
    # Проверяем длины списков
    if len(rows_indices) != len(new_values):
        raise ValueError("Количество индексов строк и новых значений должно совпадать.")

    # Определяем текущий тип данных в столбце
    current_dtype = df[column_name].dtype

    for idx, value in zip(rows_indices, new_values):
        try:
            # Преобразовываем значение к нужному типу данных
            converted_value = current_dtype.type(value)

            # Обновляем значение в дате
            df.at[idx, column_name] = converted_value
        except Exception as e:
            raise ValueError(f"Произошла ошибка при попытке преобразования значения для строки {idx}: {e}")

#Задаём строки и значения
rows_to_update = [78, 10]  # Индексы строк, которые будем обновлять
new_times = [3.0, 4]    # Новые значения для этих строк

# Вызов функции для изменения нескольких строк
replace_values_in_multiple_rows(modified_min_time_warm, rows_to_update, 'min', new_times)

modified_min_time_warm.iloc[78]

,78
depart,Нарьян-Мар
arrival,Мезень
min,3.0


# Сравнение двух матриц гравитационных сил

In [ ]:
df_inf_new = pd.DataFrame(updated_data)

# Нормализуем данные
normalized_df_inf_new = normalize_columns(df_inf_new)

# Суммируем нормализованные значения
summed_values_new = normalized_df_inf_new.iloc[:, 1:].sum(axis=1) #Все строки и колонки кроме первых

# Добавляем итоговую сумму к исходному DataFrame
result_df_inf_new = df_inf_new.copy()
result_df_inf_new['ind'] = summed_values_new

result_df_inf_new = result_df_inf_new[[result_df_inf_new.columns[0], result_df_inf_new.columns[-1]]] #Выбираем только столбцы с названием и итоговой оценкой

df_towns = pd.read_csv('main_towns.csv', delimiter = ';') #Читаем файл с опорными городами
combined_df_new = pd.concat([result_df_inf_new, df_towns])

In [ ]:
# Чтение файлов с уровнем инфраструктурной обеспеченности и данными по времени
settl = dict(zip(combined_df_new['name'], combined_df_new['ind']))

# Получение данных из min_time_warm
connect = modified_min_time_warm.values.tolist()

# Вычисления и создание нового датафрейма
Gravity = []
for row in connect:
    try:
        F1 = float(settl[row[0]])  # Значение F1 для первого населенного пункта
        F2 = float(settl[row[1]])  # Значение F2 для второго населенного пункта
        r = float(row[2])          # Расстояние между пунктами
        if r == 0:
            LocalGravity = 0
        else:
            LocalGravity = (F1 - F2) / (r ** 2)  # Рассчитываем LocalGravity
        Gravity.append([
            row[0],           # Название первого населенного пункта
            row[1],           # Название второго населенного пункта
            LocalGravity,
            F1 - F2,          # Разница между F1 и F2
            row[2],           # Время из столбца min_time_warm
            F1,               # Добавление F1
            F2                # Добавление F2
        ])                    # Добавляем F1-F2 и r
    except KeyError:
        continue  # Пропускаем строки, если ключ не найден

# Создаем новый датафрейм с нужными столбцами
local_gravity_new = pd.DataFrame(
    Gravity,
    columns=[
        'source',             # Название первого населенного пункта
        'recipient',          # Название второго населенного пункта
        'LocalGravity',
        'F1-F2',              # Разница между F1 и F2
        'time',               # Время из столбца min_time_warm
        'F1',                 # Значение F1 для деревни source
        'F2'                  # Значение F2 для деревни recipient
    ]
)

# Сортируем по LocalGravity в порядке убывания
#local_gravity_new.sort_values(by='LocalGravity', ascending=False)

In [ ]:
# Чтение CSV файла
df_row_new = local_gravity_new

# Применяем функцию ко всем строкам
df_row_new = df_row_new.apply(process_row, axis=1)
#df_row_new.sort_values(by='LocalGravity', ascending=False)

# Отображение изменённого датафрейма
#df_row_new

In [ ]:
anchor_points = ['Несь', 'Нижняя Пеша', 'Оксино', 'Лабожское', 'Великовисочное', 'Чижа', 'Усть-Кара', 'Хорей-Вер', 'Волоковая']  # Список опорных пунктов

# Нормализация разницы F1-F2 и времени связности
df_row_new['normalized_f_diff'] = (df_row_new['F1-F2'] - df_row_new['F1-F2'].min()) / (df_row_new['F1-F2'].max() - df_row_new['F1-F2'].min())

# Обратная нормализация времени (минимум = 1, максимум = 0)
df_row_new['normalized_time'] = 1 - ((df_row_new['time'] - df_row_new['time'].min()) / (df_row_new['time'].max() - df_row_new['time'].min()))

# Настройки порогов для классификации
f_diff_threshold = 0.4  # Порог для разницы F1-F2 (нормированное значение)
time_threshold = 0.6  # Порог для времени связности (нормированное значение)

# Классификация ситуаций
df_row_new['action'] = ''

for index, row in df_row_new.iterrows():
    f_diff = row['normalized_f_diff']
    time = row['normalized_time']

    if row['recipient'] in anchor_points:
        action = 'Приоритетная связь'  # Связь с опорным пунктом имеет наивысший приоритет
    elif f_diff > f_diff_threshold and time > time_threshold:
        action = 'Добавить сервисы'  # Улучшение инфраструктуры рекомендуется, если разница F1-F2 велика и время связи малое
    elif f_diff < f_diff_threshold and time < time_threshold:
        action = 'Сократить время'  # Сокращение времени путешествия рекомендуется, если разница F1-F2 мала и время связи велико
    elif f_diff >= f_diff_threshold and time <= time_threshold:
        action = 'Добавить сервисы и сократить время'  # Сокращение времени путешествия рекомендуется, если разница F1-F2 велика и время связи велико
    else:
        action = 'Никаких действий не требуется'  # Никаких действий не требуется, если разница F1-F2 и время связи находятся в приемлемых пределах

    df_row_new.at[index, 'action'] = action

# Удаляем лишние столбцы и оставляем только recipient, source и action
final_dataframe_new = df_row_new[['recipient', 'source', 'action']]

# Сохранение отчета
#final_dataframe.to_csv('recommendations_final.csv', index=False)

In [ ]:
counted_df_new = final_dataframe_new.groupby(['recipient', 'action']).size().reset_index(name='count')

final_dataframe_new = counted_df_new.sort_values(by='count', ascending=False)
final_dataframe_new

,recipient,action,count
0,Амдерма,Добавить сервисы,1
1,Андег,Добавить сервисы,1
2,Андег,Никаких действий не требуется,1
3,Бугрино,Добавить сервисы,1
4,Великовисочное,Приоритетная связь,1
5,Верхняя Пеша,Никаких действий не требуется,1
6,Вижас,Добавить сервисы,1
7,Вижас,Никаких действий не требуется,1
8,Волонга,Никаких действий не требуется,1
9,Выучейский,Добавить сервисы,1
